In [20]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import ISLP

# Load a well-known dataset (e.g., Boston housing dataset, as a substitute for Auto, for demonstration)
df = ISLP.load_data('Boston')
df['medv']

# Check the first few rows
print(df.head())

      crim    zn  indus  chas    nox     rm   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

   lstat  medv  
0   4.98  24.0  
1   9.14  21.6  
2   4.03  34.7  
3   2.94  33.4  
4   5.33  36.2  


In [21]:
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

In [32]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Calculate VIF for all features
initial_vif = calculate_vif(df.drop(columns='medv'))

initial_vif

,Feature,VIF
0,crim,2.077944
1,zn,2.842980
2,indus,14.446693
3,chas,1.148528
4,nox,73.077558
5,rm,76.160431
6,age,21.350797
7,dis,14.576652
8,rad,14.732498
9,tax,61.225416


In [22]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Function to calculate VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# Stepwise selection with integrated VIF check
def stepwise_selection_with_vif(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, vif_threshold=5.0, verbose=True):
    included = list(initial_list)
    while True:
        changed = False

        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            # Add feature temporarily to check VIF
            included.append(best_feature)
            if max(calculate_vif(X[included])['VIF']) < vif_threshold:
                changed = True
                if verbose:
                    print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
            else:
                # Remove if VIF is too high
                included.remove(best_feature)
                if verbose:
                    print('Skip {:30} due to high VIF'.format(best_feature))

        # Backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if max(calculate_vif(X[included])['VIF']) < vif_threshold:
                changed = True
                if verbose:
                    print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
            else:
                # Re-add if removal causes VIF issues
                included.append(worst_feature)

        if not changed:
            break

    return included

In [23]:
# Example usage with some dataset
X = df.drop(columns='medv')
y = df['medv']
selected_features = stepwise_selection_with_vif(X, y)
print('Selected features:', selected_features)

C:\Users\dansc\AppData\Local\Temp\ipykernel_18064\761999109.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


ValueError: zero-size array to reduction operation maximum which has no identity